In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cross_validation import train_test_split
import seaborn as sns
%matplotlib inline

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
Survived=train.Survived
train1=train.copy(deep=True)
train.drop('Survived',axis=1,inplace=True)

In [4]:
combined=[train,test]

In [5]:
for dataset in combined:
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)

In [6]:
train.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
for dataset in combined:
    dataset['Sex'].replace(['male','female'],[0,1],inplace=True)
    embarked_dummies = pd.get_dummies(dataset['Embarked'],prefix='Embarked')
    dataset = pd.concat([dataset,embarked_dummies],axis=1)
    dataset.drop('Embarked',axis=1,inplace=True)


In [8]:
#creating new feature title
for dataset in combined:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
for dataset in combined:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir',
                                                 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# map the title to int64
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combined:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)


In [9]:
#dropping Name,passengerid,ticket,cabin from train and name,ticket,cabin from test as those features seemed irrelevant
train = train.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], axis=1)
test = test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [10]:
#creating anew feature AgeBand which will help in categorizing Age into different groups.
train['AgeBand'] = pd.cut(train['Age'], 5)

In [11]:
for dataset in [train,test]:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']

In [12]:
#remove AgeBand
train = train.drop(['AgeBand'], axis=1)

In [13]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,3,0,1.0,1,0,7.2500,S,1
1,1,1,2.0,1,0,71.2833,C,3
2,3,1,1.0,0,0,7.9250,S,2
3,1,1,2.0,1,0,53.1000,S,3
4,3,0,2.0,0,0,8.0500,S,1


In [14]:
##converting categorical variable into numerical variable
for dataset in [train,test]:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [15]:
pid=test.PassengerId
test.drop(['PassengerId'],axis=1)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,3,0,2.0,0,0,7.8292,2,1
1,3,1,2.0,1,0,7.0000,0,3
2,2,0,3.0,0,0,9.6875,2,1
3,3,0,1.0,0,0,8.6625,0,1
4,3,1,1.0,1,1,12.2875,0,3
5,3,0,0.0,0,0,9.2250,0,1
6,3,1,1.0,0,0,7.6292,2,2
7,2,0,1.0,1,1,29.0000,0,1
8,3,1,1.0,0,0,7.2292,1,3
9,3,0,1.0,2,0,24.1500,0,1


In [16]:
#categorizing Fare
for dataset in [train,test]:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [17]:
train_x,test_x,train_y,test_y=train_test_split(train,Survived,train_size=0.9)

In [18]:
pid=test.PassengerId
test = test.drop(['PassengerId'], axis=1)

In [19]:
##model
x_size = train_x.shape[1]; # number of features
y_size = 1; # binary variable
n_n_fc1 = 128; # number of neurons of first layer
n_n_fc2 = 128; # number of neurons of second layer
n_n_fc3 = 128; # number of neurons of third layer

# variables for input and output 
x_data = tf.placeholder('float', shape=[None, x_size])
y_data = tf.placeholder('float', shape=[None, y_size])

# 1.layer: fully connected
W_fc1=tf.get_variable("W_fc1", shape=[x_size, n_n_fc1],initializer=tf.contrib.layers.xavier_initializer())
b_fc1 = tf.Variable(tf.constant(0.0, shape = [n_n_fc1]))  
h_fc1 = tf.nn.relu(tf.matmul(x_data, W_fc1) + b_fc1)

# dropout
tf_keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, tf_keep_prob)

# 2.layer: fully connected
W_fc2=tf.get_variable("W_fc2", shape=[n_n_fc1, n_n_fc2],initializer=tf.contrib.layers.xavier_initializer())
b_fc2 = tf.Variable(tf.constant(0.0, shape = [n_n_fc2]))  
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2) 

# dropout
h_fc2_drop = tf.nn.dropout(h_fc2, tf_keep_prob)

# 3.layer: fully connected 
W_fc3=tf.get_variable("W_fc3", shape=[n_n_fc2, n_n_fc3],initializer=tf.contrib.layers.xavier_initializer())
b_fc3 = tf.Variable(tf.constant(0.0, shape = [n_n_fc3]))  
h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3) 

# dropout
h_fc3_drop = tf.nn.dropout(h_fc3, tf_keep_prob)

# 3.layer: fully connected 
W_fc4=tf.get_variable("W_fc4", shape=[n_n_fc3, y_size],initializer=tf.contrib.layers.xavier_initializer())
b_fc4 = tf.Variable(tf.constant(0.0, shape = [y_size]))  
z_pred = tf.matmul(h_fc3_drop, W_fc4) + b_fc4  

# cost function
cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_data, logits=z_pred));

# optimisation function
tf_learn_rate = tf.placeholder(dtype='float', name="tf_learn_rate")
train_step = tf.train.AdamOptimizer(tf_learn_rate).minimize(cross_entropy)

# evaluation
y_pred = tf.nn.sigmoid(z_pred);
y_pred_class = tf.cast(tf.greater(y_pred, 0.5),'float')
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred_class, y_data ), 'float'))

# start TensorFlow session and initialize global variables
sess = tf.InteractiveSession() 
sess.run(tf.global_variables_initializer())  

keep_prob = 0.33; # dropout regularization with keeping probability
learn_rate_range = [0.01,0.005,0.0025,0.001,0.00075,0.0005,0.00025,0.0001]
learn_rate_step = 200;

x_train_batch = train_x.iloc[:,:].values.astype('float');
y_train_batch = train_y.iloc[:].values.reshape(train_y.shape[0],1).astype('float')

x_valid_batch = test_x.iloc[:,:].values.astype('float');
y_valid_batch = test_y.iloc[:].values.reshape(test_y.shape[0],1).astype('float')

x_test_batch = test.iloc[:,:].values.astype('float');

n_epoch = 500; # number of epochs
train_loss, train_acc, valid_loss, valid_acc = np.array([]), np.array([]), np.array([]), np.array([]);
n_step = -1;

# training model
for i in range(0,n_epoch):
    
    if i%learn_rate_step == 0:
        n_step += 1;
        learn_rate = learn_rate_range[n_step];
        print('set learnrate = ', learn_rate)
        
    sess.run(train_step, feed_dict={x_data: x_train_batch, y_data: y_train_batch, tf_keep_prob: keep_prob, 
                                    tf_learn_rate: learn_rate})
    
    if i%100==0:
        train_loss = np.append(train_loss, sess.run(cross_entropy, feed_dict={x_data: x_train_batch, 
                                                                              y_data: y_train_batch, 
                                                                              tf_keep_prob: 1.0}))
    
        train_acc = np.append(train_acc, accuracy.eval(feed_dict={x_data: x_train_batch, 
                                                                  y_data: y_train_batch, 
                                                                  tf_keep_prob: 1.0}))     
    
        valid_loss = np.append(valid_loss, sess.run(cross_entropy, feed_dict={x_data: x_valid_batch, 
                                                                              y_data: y_valid_batch, 
                                                                              tf_keep_prob: 1.0}))
    
        valid_acc = np.append(valid_acc, accuracy.eval(feed_dict={x_data: x_valid_batch, 
                                                                  y_data: y_valid_batch, 
                                                                  tf_keep_prob: 1.0}))      
    
        print('%d epoch: train/val loss = %.4f/%.4f, train/val acc = %.4f/%.4f'%(i+1,
                                                                                 train_loss[-1],
                                                                                 valid_loss[-1],
                                                                                 train_acc[-1],
                                                                                 valid_acc[-1]))

acc_DNN_train = train_acc[-1];
acc_DNN_valid = valid_acc[-1];
# prediction for test set
Y_pred_DNN = y_pred_class.eval(feed_dict={x_data: x_test_batch,tf_keep_prob: 1.0}).astype('int').flatten()

sess.close()
			
			

set learnrate =  0.01
1 epoch: train/val loss = 0.8024/0.6669, train/val acc = 0.6067/0.7000
101 epoch: train/val loss = 0.3696/0.4507, train/val acc = 0.8427/0.8778
set learnrate =  0.005
201 epoch: train/val loss = 0.3412/0.4369, train/val acc = 0.8577/0.8556
301 epoch: train/val loss = 0.3311/0.4671, train/val acc = 0.8602/0.8000
set learnrate =  0.0025
401 epoch: train/val loss = 0.3251/0.5043, train/val acc = 0.8614/0.8000


In [20]:
Y_pred_DNN

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0,

In [21]:
submission = pd.DataFrame({
        "PassengerId": pid,
        "Survived": Y_pred_DNN
    })
submission.to_csv('my_submission.csv', index=False)